In [1]:
#first import libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import os

#tensorflow
import tensorflow as tf
from tensorflow.python.keras.utils.vis_utils import plot_model

#import time
import time

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.pyplot as plt
import cv2
import random
import os

print("TensorFlow:", tf.__version__)
print("Devices:", tf.config.list_physical_devices())

TensorFlow: 2.20.0
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
!pip install ultralytics pillow matplotlib numpy opencv-python

In [4]:
from ultralytics import YOLO
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.pyplot as plt
import random
import shutil

In [ ]:
dataset_root = "waldo_dataset"

dirs = [
    f"{dataset_root}/images/train",
    f"{dataset_root}/images/val",
    f"{dataset_root}/labels/train",
    f"{dataset_root}/labels/val",
]

for d in dirs:
    os.makedirs(d, exist_ok=True)

print("Dataset folders created.")

In [ ]:
#get the images directory
image_dir = os.getcwd() + '/images/finding_waldo'

#get the background and waldo image directory
background_dir = image_dir + '/wheres_wally.jpg'
waldo_dir = image_dir + '/waldo.png'



waldo_sprite = Image.open(waldo_dir).convert("RGBA")
bg_full = Image.open(background_dir).convert("RGB")
print("Assets loaded.")

In [ ]:
IMG_W, IMG_H = 512, 512
WALDO_W, WALDO_H = 60, 100            # training Waldo size

waldo_resized = waldo_sprite.resize((WALDO_W, WALDO_H), Image.LANCZOS)

def jitter(im):
    im = ImageEnhance.Brightness(im).enhance(np.random.uniform(0.9, 1.1))
    im = ImageEnhance.Contrast(im).enhance(np.random.uniform(0.9, 1.1))
    if random.random() < 0.5:
        im = ImageOps.mirror(im)
    return im

def random_background_crop():
    # Simply resize the background to the training resolution.
    return bg_full.resize((IMG_W, IMG_H), Image.LANCZOS)

In [ ]:
def make_synthetic_waldo(image_id, split="train"):
    bg = random_background_crop()
    bg = jitter(bg)

    # random paste position
    x = random.randint(0, IMG_W - WALDO_W)
    y = random.randint(0, IMG_H - WALDO_H)

    # paste Waldo
    bg.paste(waldo_resized, (x, y), waldo_resized)

    # YOLO normalized bounding box
    x_center = (x + WALDO_W/2) / IMG_W
    y_center = (y + WALDO_H/2) / IMG_H
    w_norm = WALDO_W / IMG_W
    h_norm = WALDO_H / IMG_H

    img_path = f"{dataset_root}/images/{split}/{image_id}.jpg"
    lbl_path = f"{dataset_root}/labels/{split}/{image_id}.txt"

    # save image
    bg.save(img_path)

    # save label (class 0 = Waldo)
    with open(lbl_path, "w") as f:
        f.write(f"0 {x_center} {y_center} {w_norm} {h_norm}\n")

    return img_path

In [ ]:
yaml_text = f"""
path: {dataset_root}
train: images/train
val: images/val

names:
  0: waldo
"""

with open("waldo.yaml", "w") as f:
    f.write(yaml_text)

print("waldo.yaml created.")

In [ ]:
model = YOLO("yolov8s.pt")  # you can switch to yolov8n.pt for faster training

model.train(
    data="waldo.yaml",
    epochs=30,
    imgsz=640,
    device="mps",      # uses Apple Metal GPU
)

In [5]:
trained = YOLO("runs/detect/train/weights/best.pt")

result = trained.predict(
    source="test_waldo_image.jpg",   # OR another image
    device="mps",
    save=True
)

result


image 1/1 /Users/scmmw/Library/CloudStorage/OneDrive-UniversityofLeeds/Documents/GitHub/FindWaldo/test_waldo_image.jpg: 480x640 1 waldo, 219.7ms
Speed: 4.1ms preprocess, 219.7ms inference, 32.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /Users/scmmw/Library/CloudStorage/OneDrive-UniversityofLeeds/Documents/GitHub/FindWaldo/runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'waldo'}
 obb: None
 orig_img: array([[[245, 230, 174],
         [246, 230, 177],
         [248, 231, 180],
         ...,
         [115, 100, 108],
         [ 94,  79,  87],
         [107,  92, 100]],
 
        [[245, 229, 176],
         [246, 230, 177],
         [248, 231, 182],
         ...,
         [220, 206, 212],
         [182, 167, 175],
         [161, 147, 153]],
 
        [[246, 227, 176],
         [247, 227, 179],
         [249, 229, 181],
         ...,
         [196, 182, 186],
         [173, 159, 165],
         [162, 148, 152]],
 
        ...,
 
        [[140, 188, 206],
         [ 87, 132, 153],
         [156, 194, 218],
         ...,
         [255, 255, 193],
         [255, 255, 190],
         [255, 251, 187]],
 
        [[178, 228, 248],
         [ 88, 135, 156],
         [136, 177, 200],
         ...,
         [255, 250,